# Day 4 - Graphene!

Today we are going to apply what we have learned and explore the electronic structure of graphene.

In this morning session,
- write script to perform scf calculation for graphene
    - calculate the band structure
    - calculate the density of states
- post processing!
    - plot results

## DFTK.jl

**Goal**: Write a script that take as input:

   - lattice,
   - atomic positions,
   - input parameters (Ecut, kgrid, etc.)

In [1]:
# load dependencies
using Dates
using DFTK
using Plots

In [ ]:
function run(lattice, atoms; 
        Ecut=20, kgrid=[20, 20, 1], temperature=0.01)
    # setup of the DFT calculation
    model = 
    basis = 
    # call self_consistent_field
    scfres = self_consistent_field(basis)
    return scfres
end
results = run(lattice, atoms)
# plotting
plot_bandstructure(scfres, kline_density=10)
plot_dos(scfres)
heatmap(scfres.ρ[:, :, 1, 1], c=:sunset)

In [2]:
# a simple solution
function run(lattice, atoms; Ecut=20, kgrid=[20, 20, 1], tol=1e-10, temperature=0.01)
    @info "Building DFT model"
    model = model_LDA(lattice, atoms, temperature=temperature)
    @info "Constructing plane-wave basis set"
    basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid)
    @info "Solving Kohn-Sham equations self-consistently"
    println()
    println("Starting calculation at $(Dates.now())")
    println()
    println("no. of electrons    = $(model.n_electrons)")
    println("k grid              = $(kgrid)")
    println("temperature         = $(temperature)")
    println("Ecut                = $(Ecut)")
    @time scfres = self_consistent_field(basis; tol=tol)
    println()
    println("Finished calculation at $(Dates.now())")
    println()
    return basis, scfres
end;

In [3]:
# lattice parameters
a = 4.6111 # bohr 
c = 18.870 # bohr

# lattice for graphene
lattice = [[a    -0.5*a         0]; 
           [0     0.5*sqrt(3)*a 0]; 
           [0     0             c]];
# two carbon positions
positions = [[1/3., 2/3., 0], [2/3., 1/3., 0]];

# carbon
C = ElementPsp(:C, psp=load_psp(:C, functional="lda"));

# atoms
atoms = [C => positions];

In [4]:
basis, scfres = run(lattice, atoms);

┌ Info: Building DFT model
└ @ Main In[2]:3
┌ Info: Constructing plane-wave basis set
└ @ Main In[2]:5



Starting calculation at 2021-08-31T09:23:31.248

no. of electrons    = 8


┌ Info: Solving Kohn-Sham equations self-consistently
└ @ Main In[2]:7


k grid              = [20, 20, 1]
temperature         = 0.01
Ecut                = 20
n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -11.28755148401         NaN   2.62e-01   0.80    3.3
  2   -11.29647466233   -8.92e-03   3.78e-02   0.80    2.2
  3   -11.29708350461   -6.09e-04   8.43e-03   0.80    5.7
  4   -11.29711452579   -3.10e-05   9.33e-04   0.80    4.3
  5   -11.29711480798   -2.82e-07   9.05e-05   0.80    4.6
  6   -11.29711481357   -5.59e-09   5.82e-06   0.80    4.5
  7   -11.29711481357   -3.32e-12   6.13e-07   0.80    4.1
157.436381 seconds (131.26 M allocations: 17.769 GiB, 3.09% gc time, 39.94% compilation time)

Finished calculation at 2021-08-31T09:26:08.815



In [10]:
ρ = heatmap(scfres.ρ[:, :, 1, 1], c=:sunset,xaxis=nothing, yaxis=nothing)
savefig(ρ, "charge.png")

In [8]:
bands = plot_bandstructure(scfres, kline_density=10)
plot!(bands, xlabel = " ", ylabel="ε - εF", ylims=(-10, 10));

Computing bands along kpath:
       Γ -> M -> K -> Γ -> A -> L -> H -> A  and  L -> M  and  K -> H


Diagonalising Hamiltonian kblocks: 100%|████████████████| Time: 0:00:32


In [9]:
dos = plot_dos(scfres)
plot!(dos, xlabel="ε", ylabel="ρ(ε)", ylim=(0, 60));

In [16]:
summary = plot(ρ, bands, dos, layout = (1, 3), size=(800, 400), title=["ρ" "ε(k)" "ρ(ε)"])
savefig(summary, "graphene_summary.pdf") 

<img src="graphene_summary.pdf" alt="drawing" width="1000" align="left"/>